# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [1]:
from IPython.display import JSON

# imports from trulens to log and get feedback on chain
from trulens_eval.keys import *
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, db, tru

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [2]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [3]:
prompt_input = 'Como adopto un perro?'

In [4]:
gpt3_response = chain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Para adoptar un perro, hay varias opciones disponibles. Puedes buscar en refugios de animales cercanos o en organizaciones de rescate de animales. Muchas de estas organizaciones tienen perfiles en línea de perros disponibles para la adopción y también programan eventos de adopción en persona.\n\nAntes de adoptar un perro, es importante tomar en cuenta algunas cosas: el tamaño y tipo de perro que se ajusta a tu estilo de vida, si tienes suficiente espacio, si tienes los recursos para cuidar al perro y si estás dispuesto a darle el tiempo y esfuerzo necesarios para cuidarlo adecuadamente.\n\nUna vez que hayas seleccionado un perro, se requerirá llenar una solicitud de adopción y generalmente se realizará una entrevista antes de que se te permita llevar al perro a casa.\n\nNo olvides que adoptar a un perro es una gran responsabilidad y que es importante asegurarse de que es la decisión correcta para ti y para el perro.'}

# Instrument chain for logging with TruLens

In [5]:
truchain: TruChain = TruChain(chain, chain_id='Chain1_ChatApplication')

In [6]:
# Instrumented chain can operate like the original:

gpt3_response = truchain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Si estás interesado en adoptar un perro, hay varias opciones disponibles. Una buen lugar para comenzar es visitando refugios y organizaciones de rescate en tu área. Estos lugares suelen tener perros de todas las edades y razas que necesitan un hogar amoroso. También hay sitios web y aplicaciones de adopción de mascotas que pueden conectarte con perros que necesitan un hogar. \n\nAntes de adoptar un perro, es importante que consideres si tienes el tiempo, espacio y recursos para cuidar adecuadamente de tu mascota. También es esencial que familiarices con la raza de perro que te interesa para asegurarte de que sea adecuada para tus necesidades y estilo de vida. \n\nEn el proceso de adopción, es probable que se te solicite llenar una solicitud y ser sometido a una entrevista. Es posible que también se te pida que proporciones referencias y que te sometas a una verificación de antecedentes. Esto es común en los lugares de adopción de mascotas, 

In [7]:
# But can also produce a log or "record" of the execution of the chain:

gpt3_response, record = truchain.call_with_record(prompt_input)

JSON(record)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


<IPython.core.display.JSON object>

In [8]:
# We can log the records but first we need to log the chain itself:

tru.add_chain(chain_json=truchain.json)

In [9]:
# Now the record:

tru.add_record(
    prompt=prompt_input, # prompt input
    response=gpt3_response['text'], # LLM response
    record_json=record # record is returned by the TruChain wrapper
)

# Note that the `add_record` call automatically sets the `record_id` field of the
# `record_json` to the returned record id. Retrieving it from the output of `add_record` is not 
# necessary.

'record_hash_723b5673553f3a40aa8164ae8bbdb5f0'

In [10]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

*** Creating new Endpoint singleton instance for name = huggingface ***
*** Creating huggingface endpoint ***


huggingface api: 0requests [00:00, ?requests/s]

*** Creating new TP singleton instance for name = None ***


In [11]:
# This might take a moment if the public api needs to load the language model
# used in the feedback function:
feedback_result = f_lang_match.run_on_record(
    chain_json=truchain.json, record_json=record
)

In [12]:
JSON(feedback_result)

<IPython.core.display.JSON object>

In [13]:
# Alternatively, run a collection of feedback functions:

feedback_results = tru.run_feedback_functions(
    record_json=record,
    feedback_functions=[f_lang_match]
)

In [14]:
display(feedback_results)

[{'_success': True,
  'feedback_id': 'feedback_hash_26c84c76d907d808c36028e024af63c0',
  'record_id': 'record_hash_723b5673553f3a40aa8164ae8bbdb5f0',
  'language_match': 0.005036122314777458}]

In [15]:
# These can be logged:

tru.add_feedbacks(feedback_results)

## Run the TruLens dashboard to explore the quality of your LLM chain

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

## Automatic Logging

The above logging and feedback function evaluation steps can be done by TruChain.

In [ ]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    db=db
)

# Note: providing `db: TruDB` causes the above constructor to log the wrapped chain in the database specified.

# Note: any `feedbacks` specified here will be evaluated and logged whenever the chain is used.

In [ ]:
truchain("This will be automatically logged.")

## Out-of-band Feedback evaluation

In the above example, the feedback function evaluation is done in the same process as the chain evaluation. The alternative approach is the use the provided persistent evaluator started via `tru.start_deferred_feedback_evaluator`. Then specify the `feedback_mode` for `TruChain` as `deferred` to let the evaluator handle the feedback functions.

For demonstration purposes, we start the evaluator here but it can be started in another process.

In [ ]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    db=db,
    feedback_mode="deferred"
)

In [ ]:
tru.start_deferred_feedback_evaluator()

In [ ]:
truchain("This will be logged by deferred evaluator.")

Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.